## Importing required libraries

In [1]:
!pip install transformers seqeval[gpu]

  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [18 lines of output]
  Traceback (most recent call last):
    File "<string>", line 2, in <module>
    File "<pip-setuptools-caller>", line 34, in <module>
    File "C:\Users\Administrator\AppData\Local\Temp\pip-install-6fqiepgh\tensorflow-gpu_feaf57065a864c97996b3415c86aef10\setup.py", line 37, in <module>
      raise Exception(TF_REMOVAL_WARNING)
  Exception:
  
  The "tensorflow-gpu" package has been removed!
  
  Please install "tensorflow" instead.
  
  Other than the name, the two packages have been identical
  since TensorFlow 2.1, or roughly since Sep 2019. For more
  information, see: pypi.org/project/tensorflow-gpu
  
  
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tensorflow-gpu
  error: subprocess-exited-with-error
  
  Running setup.py install for tensorflow-gpu


  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for tensorflow-gpu
Failed to build tensorflow-gpu
  Running setup.py install for tensorflow-gpu: started
  Running setup.py install for tensorflow-gpu: finished with status 'error'


In [2]:
!pip install unidecode

In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForTokenClassification, BertTokenizerFast, BertConfig, BertModel

In [4]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


## Importing dataset

In [5]:
data = pd.read_csv("./train.csv", encoding='unicode_escape',nrows=1500000)
data.head()

,ABSTRACT_ID,TEXT,LOCATION,LABEL
0,14145090,velvet antlers vas are commonly used in tradit...,63,transverse aortic constriction
1,1900667,the clinical features of our cases demonstrate...,85,hodgkins lymphoma
2,8625554,ceftobiprole bpr is an investigational cephalo...,90,methicillinsusceptible s aureus
3,8157202,we have taken a basic biologic RPA to elucidat...,26,parathyroid hormonerelated protein
4,6784974,lipoperoxidationderived aldehydes for example ...,157,lipoperoxidation


## Select labels having frequency >=145

In [6]:
d = pd.DataFrame(data.LABEL.value_counts())
d = d.loc[d.LABEL >= 145]
d = list(d.index)
len(d)

689

In [7]:
data = data.loc[data['LABEL'].isin(d)]

In [8]:
data.shape

(102262, 4)

## Create new file for processed data

In [9]:
f = open("data.csv", "w")
f.write('sentence_id,word,label\n')

23

## Remove special characters

In [10]:
import re
def remove_special_characters(word):
  # w=re.split("[\\,;&|+%$!~.:/%^#\-@?\'\"(){}\[\]]",word)
  w=re.sub("[\W+\_]", "",word,0,re.IGNORECASE)
  new_word="".join(w)
  return new_word

## Removing accents

In [11]:
def remove_accents(word):
  new_word= unidecode.unidecode(word)
  return new_word

In [12]:
import re
import unidecode
index = 0
tag = 'O'
for A_ID,text,location,label in np.array(data):
  Words = text.split()
  word_index = 0
  for word in Words:
    word=remove_special_characters(word)
    word=remove_accents(word)
    if word_index == location:
      f.write(f'Sentence:{index},{word},{label}\n')
    else:
      f.write(f'Sentence:{index},{word},{tag}\n')
    word_index += 1
    f.write('\n')
  index += 1

In [13]:
f.close()

In [14]:
data2 = pd.read_csv('data.csv')

In [15]:
data2.count()

sentence_id    18061440
word           18061000
label          18061440
dtype: int64

In [16]:
print("Number of unique labels: {}".format(len(data2.label.unique())))
frequencies = data2.label.value_counts()
frequencies

Number of unique labels: 690


O                             17959178
type ii collagen                   164
hexosaminidase                     163
envelope                           162
adenocarcinoma of the lung         161
                                ...   
food deprivation                   145
motivational interviewing          145
h after inoculation                145
water immersion                    145
galactomannan                      145
Name: label, Length: 690, dtype: int64

In [17]:
labels = {}
for label, count in zip(frequencies.index, frequencies):
    if label != "O":
        if label not in labels.keys():
            labels[label] = count
        else:
            labels[label] += count
    continue

print(sorted(labels.items(), key=lambda x: x[1], reverse=True))

[('type ii collagen', 164), ('hexosaminidase', 163), ('envelope', 162), ('adenocarcinoma of the lung', 161), ('betahemolytic streptococci', 161), ('graft', 160), ('ornithine decarboxylase activity', 160), ('human endothelial cells', 159), ('kidney cells', 159), ('cardiac tamponade', 159), ('pulmonary metastases', 159), ('unbound fraction', 158), ('nomeganitrolarginine', 158), ('scanning tunneling microscope', 158), ('chemoembolization', 158), ('rnaprotein', 158), ('melting points', 158), ('left pulmonary artery', 157), ('total gastrectomy', 157), ('covalent binding', 157), ('immunohistochemical analysis', 157), ('extent of absorption', 157), ('extracellular matrix components', 157), ('adrenal medulla', 156), ('major histocompatibility complex class i', 156), ('tributyrin', 156), ('working memory', 156), ('pharyngoesophageal', 156), ('primary progressive', 156), ('parietal cortex', 156), ('cerebral perfusion', 156), ('olfactory bulbs', 155), ('opsonized zymosan', 155), ('serumfree media

## Labels to IDs

In [18]:
labels_to_ids = {k: v for v, k in enumerate(data2.label.unique())}
ids_to_labels = {v: k for v, k in enumerate(data2.label.unique())}
labels_to_ids

{'O': 0,
 'hydroxybenzoate': 1,
 'aortic valve disease': 2,
 'nomifensine': 3,
 'rnaprotein': 4,
 'socioeconomic level': 5,
 'benzoic acid': 6,
 'glatiramer acetate': 7,
 'transsphenoidal surgery': 8,
 'wall stress': 9,
 'systemic': 10,
 'parietal cortex': 11,
 'dipeptidyl peptidase': 12,
 'high protein diet': 13,
 'dietinduced obesity': 14,
 'inferior olive': 15,
 'typical development': 16,
 'graft': 17,
 'neurosecretory': 18,
 'endothelial cell monolayers': 19,
 'quantitative polymerase chain reaction': 20,
 'intragastric': 21,
 'sympathectomized': 22,
 'extracellular matrix components': 23,
 'leukemia cells': 24,
 'pseudopregnancy': 25,
 'subretinal fluid': 26,
 'adrenal medulla': 27,
 'interstitial cystitis': 28,
 'methyl methanesulfonate': 29,
 'highly polymorphic': 30,
 'oxidized': 31,
 'multislice ct': 32,
 'pars plana vitrectomy': 33,
 'dim light': 34,
 'diazoxide': 35,
 'interacting protein': 36,
 'peak force': 37,
 'tetrahydrofuran': 38,
 'alveolar bone loss': 39,
 'methyl or

In [19]:
data2

,sentence_id,word,label
0,Sentence:0,the,O
1,Sentence:0,xlnd,O
2,Sentence:0,gene,O
3,Sentence:0,from,O
4,Sentence:0,pseudomonas,O
...,...,...,...
18061435,Sentence:102261,brain,O
18061436,Sentence:102261,vitality,O
18061437,Sentence:102261,and,O
18061438,Sentence:102261,productive,O


In [20]:
labels = list(data2.label.unique())

In [21]:
# create a new column called "sentence" which groups the words by sentence 
data2['sentence'] = data2[['sentence_id','word','label']].groupby(['sentence_id'])['word'].transform(lambda x: ' '.join(map(str, x)))
# create a new column called "word_labels" which groups the tags by sentence 
data2['word_labels'] = data2[['sentence_id','word','label']].groupby(['sentence_id'])['label'].transform(lambda x: ','.join(map(str,x)))
data2.head()

,sentence_id,word,label,sentence,word_labels
0,Sentence:0,the,O,the xlnd gene from pseudomonas alcaligenes nci...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
1,Sentence:0,xlnd,O,the xlnd gene from pseudomonas alcaligenes nci...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
2,Sentence:0,gene,O,the xlnd gene from pseudomonas alcaligenes nci...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
3,Sentence:0,from,O,the xlnd gene from pseudomonas alcaligenes nci...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
4,Sentence:0,pseudomonas,O,the xlnd gene from pseudomonas alcaligenes nci...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."


In [22]:
data2 = data2[["sentence", "word_labels"]].drop_duplicates().reset_index(drop=True)
data2.head()

,sentence,word_labels
0,the xlnd gene from pseudomonas alcaligenes nci...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
1,dilatation of the ascending aorta tends to dev...,"O,O,O,O,O,O,O,O,O,O,O,aortic valve disease,O,O..."
2,the histamineh receptor blocking properties of...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
3,rnainduced silencing complex risc is a small R...,"O,O,O,O,O,O,O,rnaprotein,O,O,O,O,O,O,O,O,O,O,O..."
4,in the first half of this century cardiovascul...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."


In [23]:
len(data2)                 

102262

In [24]:
data2.iloc[41].sentence

'three kinds of nanosilver colloids have been prepared by electrolysis of silver rod using sodium citrate solution and agno mixed with polyvinyl alcohol solution as electrolyte and applying v direct current for one hour nanosilver colloids have been investigated by means of tem absorption spectrum electrophoresis experiment and sers the particle size ranges roughly from nm to nm spheroid for sample from nm to nm spheroid for sample and from to manysided for sample featuring absorption maximum at and nm respectively the surface charge of these three kinds of colloidal silver particles is positive in order to test if these nanosilver colloids can be used for sers research the cationic molecular fuchsine basic methylene blue anionic molecular BA MO neutral molecular alcidine orange and sudan red were used it was found that these nanosilver colloids have strong sers activity furthermore the nanosilver colloids that used agno mixed with PVA solution as electrolyte has the strongest sers act

In [25]:
data2.iloc[41].word_labels

'O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,methyl orange,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O'

## Define hyperparameters

In [26]:
MAX_LEN = 320
TRAIN_BATCH_SIZE = 2
VALID_BATCH_SIZE = 1
EPOCHS = 4
MAX_GRAD_NORM = 10
LEARNING_RATE = 1e-05
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

## Tokenize the data

In [27]:
class dataset(Dataset):
  def __init__(self, dataframe,labels, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

  def __getitem__(self, index):
        # step 1: get the sentence and word labels 
        sentence = self.data.sentence[index].strip()
        word_labels = self.data.word_labels[index].split(",") 

        # step 2: use tokenizer to encode sentence (includes padding/truncation up to max length)
        # BertTokenizerFast provides a handy "return_offsets_mapping" functionality for individual tokens
        encoding = self.tokenizer(sentence,
#                              is_split_into_words=True, 
                             add_special_tokens=True,
                             return_offsets_mapping=True, 
                             max_length=self.max_len,
                             padding='max_length', 
                             truncation=True
                             )
        
        # step 3: create token labels only for first word pieces of each tokenized word
        labels = [labels_to_ids[label] for label in word_labels] 
        # code based on https://huggingface.co/transformers/custom_datasets.html#tok-ner
        # create an empty array of -100 of length max_length
        encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100
        # set only labels whose first offset position is 0 and the second is not 0
        i = 0
        for idx, mapping in enumerate(encoding["offset_mapping"]):
          if mapping[0] == 0 and mapping[1] != 0:
            # overwrite label
            encoded_labels[idx] = labels[i]
            i += 1

        # step 4: turn everything into PyTorch tensors
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['labels'] = torch.as_tensor(encoded_labels)
        
        return item

  def __len__(self):
        return self.len


## Define train-test split of data

In [28]:
train_size = 0.75
train_dataset = data2.sample(frac=train_size,random_state=105)
test_dataset = data2.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(data2.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = dataset(train_dataset,labels, tokenizer, MAX_LEN)
testing_set = dataset(test_dataset,labels, tokenizer, MAX_LEN)

FULL Dataset: (102262, 2)
TRAIN Dataset: (76696, 2)
TEST Dataset: (25566, 2)


In [29]:
data2.drop(data2.index)

,sentence,word_labels


In [30]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

## Import model 

In [31]:
model = BertForTokenClassification.from_pretrained('bert-base-cased', num_labels=len(labels_to_ids))
model.to(device)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [32]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

## Defining the training function

In [33]:
# Defining the training function on the 80% of the dataset for tuning the bert model
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    
    for idx, batch in enumerate(training_loader):
        
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=labels)
        loss = outputs["loss"]
        tr_logits = outputs["logits"]
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
        
        # print(idx)
        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
#         active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

## Training the model

In [34]:
for epoch in range(EPOCHS):
    print(f"------------------------------------{epoch}-----------------------------------")
    train(epoch)

------------------------------------0-----------------------------------
Training loss per 100 training steps: 6.7485809326171875
Training loss per 100 training steps: 1.469690045175871
Training loss per 100 training steps: 0.8731442134067491
Training loss per 100 training steps: 0.6452785111976125
Training loss per 100 training steps: 0.555087510315223
Training loss per 100 training steps: 0.4853710846793271
Training loss per 100 training steps: 0.4539186551596678
Training loss per 100 training steps: 0.4157874948675937
Training loss per 100 training steps: 0.3917693701426216
Training loss per 100 training steps: 0.3854614048245281
Training loss per 100 training steps: 0.3570687947310991
Training loss per 100 training steps: 0.3442606859287242
Training loss per 100 training steps: 0.33080853751707795
Training loss per 100 training steps: 0.31296894673671827
Training loss per 100 training steps: 0.3055047248790957
Training loss per 100 training steps: 0.2962452423475561
Training loss p

## Validate the model

In [35]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
          ids = batch['input_ids'].to(device, dtype = torch.long)
          mask = batch['attention_mask'].to(device, dtype = torch.long)
          labels = batch['labels'].to(device, dtype = torch.long)
          
          outputs = model(input_ids=ids, attention_mask=mask, labels=labels)
          loss = outputs["loss"]
          eval_logits = outputs["logits"]
          eval_loss += loss.item()
          

          nb_eval_steps += 1
          nb_eval_examples += labels.size(0)
      
          if idx % 100==0:
              loss_step = eval_loss/nb_eval_steps
              print(f"Validation loss per 100 evaluation steps: {loss_step}")
            
          # compute evaluation accuracy
          flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
          active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
          flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
          
          # only compute accuracy at active labels
          active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
      
          labels = torch.masked_select(flattened_targets, active_accuracy)
          predictions = torch.masked_select(flattened_predictions, active_accuracy)
          
          eval_labels.extend(labels)
          eval_preds.extend(predictions)
          
          tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
          eval_accuracy += tmp_eval_accuracy

    labels = [[ids_to_labels[id.item()] for id in eval_labels]]
    predictions = [[ids_to_labels[id.item()] for id in eval_preds]]
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions

In [36]:
labels, predictions = valid(model, testing_loader)

Validation loss per 100 evaluation steps: 1.168244216387393e-05
Validation loss per 100 evaluation steps: 0.13412150904441028
Validation loss per 100 evaluation steps: 0.16247626487168856
Validation loss per 100 evaluation steps: 0.22640377243597956
Validation loss per 100 evaluation steps: 0.2642820271459081
Validation loss per 100 evaluation steps: 0.2383593474337086
Validation loss per 100 evaluation steps: 0.20557518551231624
Validation loss per 100 evaluation steps: 0.17637213623814008
Validation loss per 100 evaluation steps: 0.17246732343368337
Validation loss per 100 evaluation steps: 0.17125392522954658
Validation loss per 100 evaluation steps: 0.1569979724415264
Validation loss per 100 evaluation steps: 0.1490759808252324
Validation loss per 100 evaluation steps: 0.14292974923054386
Validation loss per 100 evaluation steps: 0.146244721324783
Validation loss per 100 evaluation steps: 0.13643356720568453
Validation loss per 100 evaluation steps: 0.14320098585479243
Validation l

## Classification report

In [37]:
from seqeval.metrics import classification_report,f1_score

print(classification_report(labels, predictions))

                                              precision    recall  f1-score   support

                        dipeptidyl peptidase       0.75      1.00      0.86         6
                   motivational interviewing       1.00      1.00      1.00         4
                                hemodialyzed       0.00      0.00      0.00         1
                  anaplastic lymphoma kinase       1.00      1.00      1.00         3
                          neurocysticercosis       1.00      0.71      0.83         7
                             position effect       0.00      0.00      0.00         2
                                       lewis       0.00      0.00      0.00         2
                  aminoglycoside antibiotics       1.00      1.00      1.00         6
                melaninconcentrating hormone       1.00      0.78      0.88         9
                             assisted living       0.00      0.00      0.00         1
      quantitative polymerase chain reaction       0.

In [38]:
import os

directory = "./model_2"

if not os.path.exists(directory):
    os.makedirs(directory)

# save vocabulary of the tokenizer
tokenizer.save_vocabulary(directory)
# save the model weights and its configuration file
model.save_pretrained(directory)
print('All files saved')

All files saved
